# Get Sentinel-2 L2A Data using Microsoft Planetary Computer

This notebook downloads Sentinel-2 L2A data from Microsoft Planetary Computer to match the S1S2 Water dataset.

In [2]:
import json
from pathlib import Path
from datetime import datetime, timedelta
import numpy as np

import pandas as pd
from pyproj import Transformer
import matplotlib.pyplot as plt

from tqdm.auto import tqdm

import zipfile
import rasterio as rio
from rasterio.windows import from_bounds
import urllib.request

import pystac_client
import planetary_computer

In [3]:
data_dir = Path("/media/nick/4TB Working 6/Datasets/S1S2-Water")
data_dir.mkdir(exist_ok=True, parents=True)

## Download S1S2 Water Dataset from Zenodo

In [4]:
download_urls = [
    "https://zenodo.org/records/11278238/files/catalog.json",
    "https://zenodo.org/records/11278238/files/part1.zip",
    "https://zenodo.org/records/11278238/files/part2.zip",
    "https://zenodo.org/records/11278238/files/part3.zip",
    "https://zenodo.org/records/11278238/files/part4.zip",
    "https://zenodo.org/records/11278238/files/part5.zip",
    "https://zenodo.org/records/11278238/files/part6.zip",
]

In [5]:
for url in tqdm(download_urls):
    zip_path = data_dir / url.split("/")[-1]
    folder_name = url.split("/")[-1].split(".")[0]
    if not zip_path.exists():
        urllib.request.urlretrieve(url, zip_path)
    if folder_name != "catalog":
        if not (data_dir / folder_name).exists():
            with zipfile.ZipFile(zip_path, "r") as zf:
                zf.extractall(data_dir)

  0%|          | 0/7 [00:00<?, ?it/s]

## Parse Product Metadata

In [6]:
json_files = list(data_dir.rglob("*meta.json"))

In [7]:
bands = ["B02", "B03", "B04", "B08"]

In [8]:
l1c_products = []
for json_path in json_files:
    if json_path.name != "catalog.json":
        with open(json_path, "r") as f:
            data = json.load(f)
        product = {
            "name": data["properties"]["s2_srcids"][0],
            "bbox": data["bbox"],
            "S1S2_number": data["id"].split("_")[1],
            "dir": json_path.parent,
        }
        l1c_products.append(product)

In [9]:
len(l1c_products)

65

## Connect to Planetary Computer STAC API

In [10]:
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)

In [11]:
def download_sentinel2_from_pc(
    catalog,
    bbox_epsg3857: list,
    capture_date: str,
    mgrs_tile: str,
    bands: list,
    output_path: Path,
):
    """
    Download Sentinel-2 L2A data from Planetary Computer.

    Args:
        catalog: pystac_client catalog
        bbox_epsg3857: Bounding box in EPSG:3857 [xmin, ymin, xmax, ymax]
        capture_date: Date string in format YYYYMMDDTHHMMSS
        mgrs_tile: MGRS tile ID (e.g., "32TQM")
        bands: List of band names to download
        output_path: Path to save the output GeoTIFF
    """
    # Convert bbox from EPSG:3857 to EPSG:4326
    transformer = Transformer.from_crs("EPSG:3857", "EPSG:4326", always_xy=True)
    x_min, y_min, x_max, y_max = bbox_epsg3857
    lon_min, lat_min = transformer.transform(x_min, y_min)
    lon_max, lat_max = transformer.transform(x_max, y_max)
    bbox_4326 = [lon_min, lat_min, lon_max, lat_max]

    # Parse the date and create a search range
    dt = datetime.strptime(capture_date, "%Y%m%dT%H%M%S")
    date_str = dt.strftime("%Y-%m-%d")
    time_range = f"{date_str}/{date_str}"

    # Search for matching items
    search = catalog.search(
        collections=["sentinel-2-l2a"],
        bbox=bbox_4326,
        datetime=time_range,
        query={"s2:mgrs_tile": {"eq": mgrs_tile}},
    )

    items = list(search.items())

    if not items:
        # Try searching with a wider date range (same day might have timezone issues)
        dt_start = dt - timedelta(days=1)
        dt_end = dt + timedelta(days=1)
        time_range = f"{dt_start.strftime('%Y-%m-%d')}/{dt_end.strftime('%Y-%m-%d')}"

        search = catalog.search(
            collections=["sentinel-2-l2a"],
            bbox=bbox_4326,
            datetime=time_range,
            query={"s2:mgrs_tile": {"eq": mgrs_tile}},
        )
        items = list(search.items())

    if not items:
        raise Exception(f"No products found for tile {mgrs_tile} on {date_str}")

    # Get the first matching item
    item = items[0]

    # Debug: print available assets
    print(f"  Found item: {item.id}")
    # print(f"  Available assets: {list(item.assets.keys())}")

    # Download and stack the bands
    arrays = []
    profile = None

    for band in bands:
        # Planetary Computer uses uppercase band names (B02, B03, etc.)
        asset_key = band
        if asset_key not in item.assets:
            raise Exception(
                f"Band {band} not found in item assets. Available: {list(item.assets.keys())}"
            )

        href = item.assets[asset_key].href

        with rio.open(href) as src:
            if profile is None:
                profile = src.profile.copy()
            # Read the full band at 10m resolution
            data = src.read(1, out_shape=(10980, 10980))
            arrays.append(data)

    # Stack and save
    arrays = np.array(arrays)
    profile.update(count=len(bands), driver="GTiff", compress="lzw")

    with rio.open(output_path, "w", **profile) as dst:
        dst.write(arrays)

    return True

## Download Sentinel-2 L2A Data

In [12]:
p_bar = tqdm(total=len(l1c_products), desc="Successful Downloads")
failed_p_bar = tqdm(total=len(l1c_products), desc="Failed Downloads")
failed_products = []

for product in l1c_products:
    l1c_id = product["name"]
    s1s2_number = product["S1S2_number"]
    bbox = product["bbox"]

    raster_name = f"sentinel12_s2_{s1s2_number}_L2A_img.tif"
    output_path = product["dir"] / raster_name

    if output_path.exists():
        p_bar.update(1)
        p_bar.refresh()
        continue

    # Parse L1C product ID to get tile and date info
    # Format: S2A_MSIL1C_YYYYMMDDTHHMMSS_Nxxxx_Rxxx_Txxxxx_YYYYMMDDTHHMMSS
    sat, sensor, capture_date, N, R, T, processing_date = l1c_id.split("_")
    mgrs_tile = T[1:]  # Remove 'T' prefix

    print(f"Processing: {raster_name} (Tile: {mgrs_tile}, Date: {capture_date})")

    try:
        download_sentinel2_from_pc(
            catalog=catalog,
            bbox_epsg3857=bbox,
            capture_date=capture_date,
            mgrs_tile=mgrs_tile,
            bands=bands,
            output_path=output_path,
        )
        p_bar.update(1)
        p_bar.refresh()
    except Exception as e:
        print(f"Failed: {e}")
        failed_products.append({"product": product, "error": str(e)})
        failed_p_bar.update(1)
        failed_p_bar.refresh()

Successful Downloads:   0%|          | 0/65 [00:00<?, ?it/s]

Failed Downloads:   0%|          | 0/65 [00:00<?, ?it/s]

In [13]:
# Print failed products summary
if failed_products:
    print(f"\nFailed to download {len(failed_products)} products:")
    for fp in failed_products:
        print(f"  - {fp['product']['name']}: {fp['error']}")

In [14]:
# Verify downloads
successful = 0
missing = 0
for product in l1c_products:
    s1s2_number = product["S1S2_number"]
    raster_name = f"sentinel12_s2_{s1s2_number}_L2A_img.tif"
    output_path = product["dir"] / raster_name
    if output_path.exists():
        successful += 1
    else:
        missing += 1

print(f"Successfully downloaded: {successful}/{len(l1c_products)}")
print(f"Missing: {missing}/{len(l1c_products)}")

Successfully downloaded: 65/65
Missing: 0/65
